In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
csv_source_file_path = 'csv_source_files/'
csv_result_file_path = 'csv_result_files/'

#load cases
source_cases = pd.read_csv(csv_source_file_path + '455fd63b-603d-4608-8216-7d8647f43350.csv')
source_cases = source_cases.dropna(subset=['Accurate_Episode_Date','Case_Reported_Date','Test_Reported_Date','Specimen_Date'])
source_cases = source_cases[(source_cases['Reporting_PHU_City']=='Toronto') | 
                            (source_cases['Reporting_PHU_City']=='Ottawa') |
                            (source_cases['Reporting_PHU_City']=='Whitby') |
                            (source_cases['Reporting_PHU_City']=='Oakville') |
                            (source_cases['Reporting_PHU_City']=='Mississauga') |
                            (source_cases['Reporting_PHU_City']=='Newmarket')]

source_cases['Case_Reported_Date'] = pd.to_datetime(source_cases['Case_Reported_Date']).dt.strftime('%Y-%m-%d')
source_cases = source_cases[ (source_cases['Case_Reported_Date']>'2020-10-31') & (source_cases['Case_Reported_Date']<'2021-03-01') ]

source_cases['Accurate_Episode_Date'] = pd.to_datetime(source_cases['Accurate_Episode_Date']).dt.strftime('%Y-%m-%d')
source_cases['Test_Reported_Date'] = pd.to_datetime(source_cases['Test_Reported_Date']).dt.strftime('%Y-%m-%d')
source_cases['Specimen_Date'] = pd.to_datetime(source_cases['Specimen_Date']).dt.strftime('%Y-%m-%d')

#load pre-processed dimensions
Date_dimension = pd.read_csv(csv_result_file_path + "Date_dimension.csv")
Mobility_dimension = pd.read_csv(csv_result_file_path + "pre_Mobility_dimension.csv")
Weather_dimension = pd.read_csv(csv_result_file_path + "pre_Weather_dimension.csv")
PHU_Location_dimension = pd.read_csv(csv_result_file_path + "PHU_Location_dimension.csv")
Patient_dimension = pd.read_csv(csv_result_file_path + "Patient_dimension.csv")



In [3]:
fact_table = pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key','Specimen_date_key',
                                   'Patient_key','PHU_Location_key','Mobility_key','Weather_key', #missing special measure 注意顺序
                                   'Resolved','Unresolved','Fatal'])

In [4]:
current_row = 0
row_length = len(source_cases)
start = time.time()

for idx, row in source_cases.iterrows():
    current_row += 1
    if(current_row%5000 == 0):
        percentage = round((current_row/row_length)*100, 2)
        seconds_spent = time.time()-start
        remaining = seconds_spent/percentage*100/60
        print("%s of %s      %s percent done\nseconds spent: %s      remaining minutes: %s\n"%
              (str(current_row), str(row_length), str(percentage),str(seconds_spent),str(remaining)))
    
    
    resolved = False
    unresolved = False
    fatal = False
    if (row["Outcome1"] == "Resolved"):
        resolved = True
    elif (row["Outcome1"] == "Not Resolved"):
        unresolved = True
    elif(row["Outcome1"] == "Fatal"):
        fatal = True
    
    use_weather_of = 'Toronto'
    if row["Reporting_PHU_City"] == 'Ottawa':
        use_weather_of = 'Ottawa'
    
    fact_table_row = [Date_dimension[ row["Accurate_Episode_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Case_Reported_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Test_Reported_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      Date_dimension[ row["Specimen_Date"]==Date_dimension['date'] ]['Date_surrogate_key'].values[0],
                      current_row - 1, #this is the Patient_key
                      PHU_Location_dimension[PHU_Location_dimension["ID"]==row["Reporting_PHU_ID"]]['PHU_Location_key'].values[0],
                      Mobility_dimension[ (row["Reporting_PHU_City"]==Mobility_dimension["Subregion"])&
                                         (row["Case_Reported_Date"]==Mobility_dimension["date"]) ]['Mobility_key'].values[0],
                      Weather_dimension[ (Weather_dimension["Station Name"]==use_weather_of)&
                                        (row["Case_Reported_Date"]==Weather_dimension["date"]) ].values[0],
                      resolved,unresolved,fatal]
    
    fact_table.loc[len(fact_table)] = fact_table_row
print("Finished with %s minutes"%((time.time()-start)/60))

5000 of 151630      3.3 percent done
seconds spent: 30.152660369873047      remaining minutes: 15.228616348420733

10000 of 151630      6.6 percent done
seconds spent: 67.12105083465576      remaining minutes: 16.94976031178176

15000 of 151630      9.89 percent done
seconds spent: 115.16153454780579      remaining minutes: 19.40706682639127

20000 of 151630      13.19 percent done
seconds spent: 176.01228547096252      remaining minutes: 22.240622374395063

25000 of 151630      16.49 percent done
seconds spent: 251.92092323303223      remaining minutes: 25.461989411060465

30000 of 151630      19.79 percent done
seconds spent: 344.28542709350586      remaining minutes: 28.994898694079996



KeyboardInterrupt: 

In [ ]:
fact_table.head()

In [ ]:
csv_result_file_path = 'csv_result_files/'
fact_table.to_csv(csv_result_file_path + 'Fact_table.csv',index=False)